In [5]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split

# Import Classifier Model
from sklearn.linear_model import LogisticRegression # Logistic Regression
from sklearn.svm import SVC # Support Vector Machine
from sklearn.naive_bayes import GaussianNB # Import Gaussian Naive Bayes
from sklearn.naive_bayes import MultinomialNB # Import Multinomial Naive Bayes
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent
from sklearn.neighbors import KNeighborsClassifier # K Nearest Neighbor
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.ensemble import RandomForestClassifier # Random Forest
from xgboost.sklearn import XGBClassifier #XGBoost

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [6]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [62]:
df_raw = pd.read_csv('../data/train.csv')

df_raw.head()

,id,timestamp,remote_addr,remote_user,hostname,time_local,request,status,body_bytes_sent,http_referer,...,upstream_connect_time,upstream_header_time,upstream_response_time,upstream_addr,upstream_status,connection,connection_requests,data.srcip,agent.ip,rule.mitre.tactic
0,0,"Aug 29, 2023 @ 17:00:03.058",114.4.220.184,-,vm-director-nginx,30/Aug/2023:00:00:02 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.001,0.002,0.002,10.45.184.148:32442,404,688306444,2,114.4.220.184,192.168.1.99,"[""non-attack""]"
1,1,"Aug 29, 2023 @ 17:00:33.070",10.200.160.148,-,vm-director-nginx,30/Aug/2023:00:00:31 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0,0.002,0.002,10.45.184.154:32442,404,688306865,2,10.200.160.148,192.168.1.99,"[""non-attack""]"
2,2,"Aug 29, 2023 @ 17:00:33.072",180.248.46.18,-,vm-director-nginx,30/Aug/2023:00:00:31 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.001,0.002,0.002,10.45.184.148:32442,404,688306855,2,180.248.46.18,192.168.1.99,"[""non-attack""]"
3,3,"Aug 29, 2023 @ 17:00:33.076",125.166.3.1,-,vm-director-nginx,30/Aug/2023:00:00:32 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0,0.002,0.002,10.45.184.154:32442,404,688306869,2,125.166.3.1,192.168.1.99,"[""non-attack""]"
4,4,"Aug 29, 2023 @ 17:00:59.118",103.108.23.13,-,vm-director-nginx,30/Aug/2023:00:00:58 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.001,0.003,0.003,10.45.184.154:32442,404,688307225,2,103.108.23.13,192.168.1.99,"[""non-attack""]"


In [63]:
def is_post_login(df):
    ret = df.copy()
    ret['is_post_login'] = ret['request'].str.contains(r'POST.*login', regex=True)
    ret = ret.drop(columns=['request'])
    return ret

def window_count(df):
    user_request = df.groupby('remote_addr')['id'].count().reset_index()
    user_request.rename(columns={'id': 'user_count'}, inplace=True)

    # Merge the transaction count back into the original DataFrame
    ret = df.merge(user_request, on='remote_addr', how='left')
    ret = ret.drop(columns=['remote_addr', 'id'])
    return ret
    
def get_mean(row):
    sum_ = 0
    for i in row:
        sum_ += float(i)
    return sum_ / len(row)

def handle_urt(df):
    ret = df.copy()
    ret['upstream_response_time'] = ret['upstream_response_time'].str.split(", ").apply(get_mean)
    return ret

def get_status_group(df):
    ret = df.copy()
    ret['status_group'] = ret['status'].str.slice(0, 1)
    ret['status_group'] = ret['status_group'].astype('object')
    ret = ret.drop(columns=['status'])
    return ret

In [64]:
df_pre = df_raw.copy()[['id',
                        'remote_addr',
                        'request',
                        'status', 
                        'body_bytes_sent', 
                        'request_method', 
                        'request_length', 
                        'bytes_sent', 
                        'request_time',
                        'upstream_response_time',
                        'connection_requests',
                        'rule.mitre.tactic'
                        ]]

df_pre = is_post_login(df_pre)
df_pre = window_count(df_pre)
df_pre = handle_urt(df_pre)

df_pre['status'] = df_pre['status'].astype('string')
df_pre = get_status_group(df_pre)

df_pre = pd.concat([df_pre, df_pre[df_pre['rule.mitre.tactic'] == '["Credential Access"]']])
df_pre = pd.concat([df_pre, df_pre[df_pre['rule.mitre.tactic'] == '["Credential Access"]']])
df_pre = pd.concat([df_pre, df_pre[df_pre['rule.mitre.tactic'] == '["Credential Access"]']])

df_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54487 entries, 0 to 22171
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   body_bytes_sent         54487 non-null  int64  
 1   request_method          54487 non-null  object 
 2   request_length          54487 non-null  int64  
 3   bytes_sent              54487 non-null  int64  
 4   request_time            54487 non-null  float64
 5   upstream_response_time  54487 non-null  float64
 6   connection_requests     54487 non-null  int64  
 7   rule.mitre.tactic       54487 non-null  object 
 8   is_post_login           54487 non-null  bool   
 9   user_count              54487 non-null  int64  
 10  status_group            54487 non-null  object 
dtypes: bool(1), float64(2), int64(5), object(3)
memory usage: 4.6+ MB


In [88]:
encode_label = {
    '["non-attack"]': 0,
    '["Impact"]': 1,
    '["Defense Evasion"]': 2,
    '["Reconnaissance"]': 3,
    '["Credential Access"]': 4,
    '["Initial Access"]': 5
}

decode_label = {
    0: '["non-attack"]',
    1: '["Impact"]',
    2: '["Defense Evasion"]',
    3: '["Reconnaissance"]',
    4: '["Credential Access"]',
    5: '["Initial Access"]'
}

In [66]:
X = df_pre.copy().drop(columns=['rule.mitre.tactic'])
y = df_pre['rule.mitre.tactic'].apply(lambda x: encode_label[x])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

numerical_features = X_train.select_dtypes(include=['number']).columns
categorical_features = X_train.select_dtypes(exclude=['number']).columns

# Encode categorical variables (e.g., one-hot encoding)
X_train = pd.get_dummies(X_train, columns=categorical_features)
X_test = pd.get_dummies(X_test, columns=categorical_features)

# Scale/Normalize numerical features (e.g., standardization)
scaler = StandardScaler()
# scaler = MinMaxScaler()
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
X_test[numerical_features] = scaler.transform(X_test[numerical_features])

In [67]:
X_train

,body_bytes_sent,request_length,bytes_sent,request_time,upstream_response_time,connection_requests,user_count,request_method_GET,request_method_HEAD,request_method_POST,is_post_login_False,is_post_login_True,status_group_2,status_group_3,status_group_4,status_group_5
15150,1.007095,-0.005576,0.996657,0.007805,0.018985,0.032657,-0.337351,0,0,1,1,0,1,0,0,0
32949,-0.235801,-0.005928,-0.237087,-0.053532,-0.044613,-0.111654,-0.353083,1,0,0,1,0,0,0,1,0
33671,0.581305,-0.005855,0.584446,1.057338,-0.040227,0.176968,0.181791,1,0,0,1,0,0,0,1,0
38635,-0.235801,-0.005862,-0.237087,-0.053532,-0.045161,-0.183810,-0.339973,1,0,0,1,0,0,0,1,0
27534,-0.235801,-0.005843,-0.237087,-0.054056,-0.045161,-0.201849,-0.360948,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17689,0.129843,-0.005096,0.130365,-0.054580,-0.045161,-0.219888,4.028160,1,0,0,1,0,0,0,1,0
13761,-0.235801,-0.005850,-0.237087,-0.054580,-0.045161,-0.201849,-0.326863,1,0,0,1,0,0,0,1,0
17183,-0.144115,-0.004695,-0.136020,0.032444,0.043108,0.014618,-0.358327,0,0,1,1,0,1,0,0,0
6701,-0.235801,-0.005855,-0.237087,-0.054056,-0.045710,-0.201849,-0.363570,1,0,0,1,0,0,0,1,0


In [68]:
# Calculate the number of samples for each label.
classes_count = df_pre['rule.mitre.tactic'].value_counts().to_numpy()

# Calculate total samples.
total = classes_count.sum()
num_classes = len(classes_count)

# Calculate the weight for each label.
weight_for_0 = (1 / classes_count[0]) * (total / num_classes)
weight_for_1 = (1 / classes_count[1]) * (total / num_classes)
weight_for_2 = (1 / classes_count[2]) * (total / num_classes)
weight_for_3 = (1 / classes_count[3]) * (total / num_classes)
weight_for_4 = (1 / classes_count[4]) * (total / num_classes)
weight_for_5 = (1 / classes_count[5]) * (total / num_classes)


class_weight = {
                0: weight_for_0,
                1: weight_for_1,
                2: weight_for_2,
                3: weight_for_3,
                4: weight_for_4,
                5: weight_for_5,
               }

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))
print('Weight for class 2: {:.2f}'.format(weight_for_2))
print('Weight for class 3: {:.2f}'.format(weight_for_3))
print('Weight for class 4: {:.2f}'.format(weight_for_4))
print('Weight for class 5: {:.2f}'.format(weight_for_5))

Weight for class 0: 0.17
Weight for class 1: 5.09
Weight for class 2: 16.19
Weight for class 3: 313.14
Weight for class 4: 1135.15
Weight for class 5: 2270.29


In [69]:
# Check model after the class being weighted

model = RandomForestClassifier(class_weight=class_weight)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Random Forest F1-Score : {f1_score(y_test, y_pred, average="weighted")}')

Random Forest F1-Score : 0.9797062989942317


In [70]:
pd.DataFrame(y_pred).value_counts()

0    10405
1      357
3      134
4        1
5        1
dtype: int64

In [ ]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'max_depth': [10, 70, 80, 100, 120, 130, 150],
    'max_features': [3, 5, 8, 10, 12, 15, 18, 20],
    'n_estimators': [100, 200, 300, 500, 700, 1000]
}
# Create a based model
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 336 candidates, totalling 1008 fits


In [113]:
best_grid = grid_search.best_estimator_
best_grid

RandomForestClassifier(class_weight={0: 0.17429927768500922,
                                     1: 5.090340059790732, 2: 16.18746286393345,
                                     3: 313.14367816091954,
                                     4: 1135.1458333333333,
                                     5: 2270.2916666666665},
                       max_depth=120, max_features=15, n_estimators=200)

In [102]:
# After tuning 1

model = RandomForestClassifier(
    class_weight=class_weight,
    max_depth=100,
    max_features=15,
    n_estimators=200,
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Random Forest F1-Score : {f1_score(y_test, y_pred, average="weighted")}')

Random Forest F1-Score : 0.9803789401825242


In [114]:
# After tuning 2

model = RandomForestClassifier(
    class_weight=class_weight,
    max_depth=120,
    max_features=15,
    n_estimators=200,
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'Random Forest F1-Score : {f1_score(y_test, y_pred, average="weighted")}')

Random Forest F1-Score : 0.9806319263391182


# Predicting Test set

In [115]:
df_raw_test = pd.read_csv('../data/test.csv')
df_raw_test

,id,timestamp,remote_addr,remote_user,hostname,time_local,request,status,body_bytes_sent,http_referer,...,request_time,upstream_connect_time,upstream_header_time,upstream_response_time,upstream_addr,upstream_status,connection,connection_requests,data.srcip,agent.ip
0,54481,"Sep 5, 2023 @ 00:26:44.850",10.26.152.242,-,vm-director-nginx,05/Sep/2023:07:26:43 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.000,0.001,0.001,10.45.184.154:32442,404,721725502,2,10.26.152.242,192.168.1.99
1,54482,"Sep 5, 2023 @ 00:26:44.880",202.80.215.96,-,vm-director-nginx,05/Sep/2023:07:26:44 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.001,0.003,0.003,10.45.184.148:32442,404,721725753,2,202.80.215.96,192.168.1.99
2,54483,"Sep 5, 2023 @ 00:26:46.898",36.85.73.24,-,vm-director-nginx,05/Sep/2023:07:26:44 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.003,0.001,0.002,0.002,10.45.184.148:32442,404,721725865,2,36.85.73.24,192.168.1.99
3,54484,"Sep 5, 2023 @ 00:26:46.945",180.248.23.157,-,vm-director-nginx,05/Sep/2023:07:26:46 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.000,0.002,0.002,10.45.184.148:32442,404,721725740,2,180.248.23.157,192.168.1.99
4,54485,"Sep 5, 2023 @ 00:26:48.856",182.1.80.255,-,vm-director-nginx,05/Sep/2023:07:26:46 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.001,0.000,0.001,0.001,10.45.184.154:32442,404,721726205,2,182.1.80.255,192.168.1.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13594,68075,"Sep 6, 2023 @ 02:12:49.915",203.78.117.29,-,vm-director-nginx,06/Sep/2023:09:12:47 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.000,0.001,0.001,10.45.184.148:32442,404,729254342,2,203.78.117.29,192.168.1.99
13595,68076,"Sep 6, 2023 @ 02:12:51.882",125.166.3.60,-,vm-director-nginx,06/Sep/2023:09:12:51 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.001,0.001,0.002,0.002,10.45.184.154:32442,404,729254818,2,125.166.3.60,192.168.1.99
13596,68077,"Sep 6, 2023 @ 02:12:51.888",10.200.16.80,-,vm-director-nginx,06/Sep/2023:09:12:51 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.000,0.002,0.002,10.45.184.154:32442,404,729254836,2,10.200.16.80,192.168.1.99
13597,68078,"Sep 6, 2023 @ 02:12:53.879",103.127.65.33,-,vm-director-nginx,06/Sep/2023:09:12:52 +0700,GET /auth/resources/ofl9o/login/ub/ HTTP/2.0,404,118,-,...,0.002,0.000,0.002,0.002,10.45.184.148:32442,404,729254963,2,103.127.65.33,192.168.1.99


In [116]:
def is_post_login(df):
    ret = df.copy()
    ret['is_post_login'] = ret['request'].str.contains(r'POST.*login', regex=True)
    ret = ret.drop(columns=['request'])
    return ret

def window_count(df):
    user_request = df.groupby('remote_addr')['id'].count().reset_index()
    user_request.rename(columns={'id': 'user_count'}, inplace=True)

    # Merge the transaction count back into the original DataFrame
    ret = df.merge(user_request, on='remote_addr', how='left')
    ret = ret.drop(columns=['remote_addr', 'id'])
    return ret
    
def get_mean(row):
    sum_ = 0
    for i in row:
        sum_ += float(i)
    return sum_ / len(row)

def handle_urt(df):
    ret = df.copy()
    ret['upstream_response_time'] = ret['upstream_response_time'].str.split(", ").apply(get_mean)
    return ret

def get_status_group(df):
    ret = df.copy()
    ret['status_group'] = ret['status'].str.slice(0, 1)
    ret = ret.drop(columns=['status'])
    return ret

In [117]:
df_pre_test = df_raw_test.copy()[['id',
                        'remote_addr',
                        'request',
                        'status', 
                        'body_bytes_sent', 
                        'request_method', 
                        'request_length', 
                        'bytes_sent', 
                        'request_time',
                        'upstream_response_time',
                        'connection_requests'
                        ]]

df_pre_test = is_post_login(df_pre_test)
df_pre_test = window_count(df_pre_test)
df_pre_test = handle_urt(df_pre_test)

df_pre_test['status'] = df_pre_test['status'].astype('string')
df_pre_test = get_status_group(df_pre_test)

df_pre_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13599 entries, 0 to 13598
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   body_bytes_sent         13599 non-null  int64  
 1   request_method          13599 non-null  object 
 2   request_length          13599 non-null  int64  
 3   bytes_sent              13599 non-null  int64  
 4   request_time            13599 non-null  float64
 5   upstream_response_time  13599 non-null  float64
 6   connection_requests     13599 non-null  int64  
 7   is_post_login           13599 non-null  bool   
 8   user_count              13599 non-null  int64  
 9   status_group            13599 non-null  string 
dtypes: bool(1), float64(2), int64(5), object(1), string(1)
memory usage: 1.1+ MB


In [118]:
df_pre_test

,body_bytes_sent,request_method,request_length,bytes_sent,request_time,upstream_response_time,connection_requests,is_post_login,user_count,status_group
0,118,GET,80,330,0.002,0.001,2,False,1,4
1,118,GET,83,330,0.002,0.003,2,False,1,4
2,118,GET,85,330,0.003,0.002,2,False,1,4
3,118,GET,80,330,0.002,0.002,2,False,1,4
4,118,GET,84,330,0.001,0.001,2,False,2,4
...,...,...,...,...,...,...,...,...,...,...
13594,118,GET,103,330,0.002,0.001,2,False,1,4
13595,118,GET,88,330,0.001,0.002,2,False,5,4
13596,118,GET,81,330,0.002,0.002,2,False,1,4
13597,118,GET,81,330,0.002,0.002,2,False,1,4


['body_bytes_sent',
 'request_length',
 'bytes_sent',
 'request_time',
 'upstream_response_time',
 'connection_requests',
 'user_count',
 'request_method_GET',
 'request_method_HEAD',
 'request_method_POST',
 'is_post_login_False',
 'is_post_login_True',
 'status_group_2',
 'status_group_3',
 'status_group_4',
 'status_group_5']

In [119]:
X_test_test = df_pre_test.copy()

numerical_features = X_test_test.select_dtypes(include=['number']).columns
categorical_features = X_test_test.select_dtypes(exclude=['number']).columns

# Encode categorical variables (e.g., one-hot encoding)
X_test_test = pd.get_dummies(X_test_test, columns=categorical_features)
X_test_test['status_group_3'] = 0
X_test_test = X_test_test[X_train.columns.tolist()]


# Scale/Normalize numerical features (e.g., standardization)
scaler = StandardScaler()
# scaler = MinMaxScaler()
X_test_test[numerical_features] = scaler.fit_transform(X_test_test[numerical_features])

X_test_test

,body_bytes_sent,request_length,bytes_sent,request_time,upstream_response_time,connection_requests,user_count,request_method_GET,request_method_HEAD,request_method_POST,is_post_login_False,is_post_login_True,status_group_2,status_group_3,status_group_4,status_group_5
0,-0.164092,-0.297010,-0.165157,-0.050539,-0.042976,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
1,-0.164092,-0.282957,-0.165157,-0.050539,-0.041687,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
2,-0.164092,-0.273588,-0.165157,-0.049908,-0.042331,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
3,-0.164092,-0.297010,-0.165157,-0.050539,-0.042331,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
4,-0.164092,-0.278273,-0.165157,-0.051171,-0.042976,-0.17342,-0.227058,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13594,-0.164092,-0.189270,-0.165157,-0.050539,-0.042976,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
13595,-0.164092,-0.259535,-0.165157,-0.051171,-0.042331,-0.17342,-0.191042,1,0,0,1,0,0,0,1,0
13596,-0.164092,-0.292325,-0.165157,-0.050539,-0.042331,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0
13597,-0.164092,-0.292325,-0.165157,-0.050539,-0.042331,-0.17342,-0.239063,1,0,0,1,0,0,0,1,0


In [108]:
test_predict = model.predict(X_test_test)
test_predict = pd.DataFrame(temp)[0].apply(lambda x: decode_label[x])

In [94]:
submission = pd.DataFrame({'id': df_raw_test.copy()['id'], 'rule.mitre.tactic':test_predict })
submission

,id,rule.mitre.tactic
0,54481,"[""non-attack""]"
1,54482,"[""non-attack""]"
2,54483,"[""non-attack""]"
3,54484,"[""non-attack""]"
4,54485,"[""non-attack""]"
...,...,...
13594,68075,"[""non-attack""]"
13595,68076,"[""non-attack""]"
13596,68077,"[""non-attack""]"
13597,68078,"[""non-attack""]"


In [109]:
submission = pd.DataFrame({'id': df_raw_test.copy()['id'], 'rule.mitre.tactic':test_predict })
submission

,id,rule.mitre.tactic
0,54481,"[""non-attack""]"
1,54482,"[""non-attack""]"
2,54483,"[""non-attack""]"
3,54484,"[""non-attack""]"
4,54485,"[""non-attack""]"
...,...,...
13594,68075,"[""non-attack""]"
13595,68076,"[""non-attack""]"
13596,68077,"[""non-attack""]"
13597,68078,"[""non-attack""]"


In [120]:
test_predict = model.predict(X_test_test)
test_predict = pd.DataFrame(temp)[0].apply(lambda x: decode_label[x])

In [121]:
submission = pd.DataFrame({'id': df_raw_test.copy()['id'], 'rule.mitre.tactic':test_predict })
submission

,id,rule.mitre.tactic
0,54481,"[""non-attack""]"
1,54482,"[""non-attack""]"
2,54483,"[""non-attack""]"
3,54484,"[""non-attack""]"
4,54485,"[""non-attack""]"
...,...,...
13594,68075,"[""non-attack""]"
13595,68076,"[""non-attack""]"
13596,68077,"[""non-attack""]"
13597,68078,"[""non-attack""]"


In [122]:
submission['rule.mitre.tactic'].value_counts()

["non-attack"]        12945
["Impact"]              653
["Initial Access"]        1
Name: rule.mitre.tactic, dtype: int64

[CV] END ....max_depth=70, max_features=10, n_estimators=300; total time=  19.6s
[CV] END ....max_depth=120, max_features=3, n_estimators=100; total time=   2.9s
[CV] END ...max_depth=120, max_features=15, n_estimators=100; total time=   7.3s
[CV] END ....max_depth=150, max_features=5, n_estimators=200; total time=   7.7s
[CV] END ....max_depth=120, max_features=3, n_estimators=200; total time=  18.3s
[CV] END ...max_depth=100, max_features=18, n_estimators=200; total time=  18.3s
[CV] END ....max_depth=10, max_features=12, n_estimators=200; total time=  21.6s
[CV] END ....max_depth=120, max_features=5, n_estimators=300; total time=   9.5s
[CV] END ....max_depth=150, max_features=5, n_estimators=100; total time=   3.3s
[CV] END ...max_depth=150, max_features=12, n_estimators=100; total time=   5.6s
[CV] END .....max_depth=70, max_features=5, n_estimators=100; total time=  20.5s
[CV] END .....max_depth=10, max_features=5, n_estimators=100; total time=   4.7s
[CV] END ...max_depth=120, m

In [124]:
submission.to_csv('submission-2.csv', index=False)